In [ ]:
#data wrangling
import numpy as np
import pandas as pd
import datetime

#plotting
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#ML libraries
#TODO: Sort out which ones we really need
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#other utilities
import os

In [ ]:
data_dir = os.fsencode('./data/')

In [ ]:
#read in he CSV files containing the data
#TODO: Abstract and build as function for util_pkg
df_dict = {}
for filename in os.listdir(data_dir):
    filepath_str = str(os.fsdecode(os.path.join(data_dir,filename)))
    df_name_str = str(os.fsdecode(os.path.splitext(filename)[0]))
    df_dict[df_name_str] = pd.read_csv(filepath_str, sep = '\t')
    print(df_name_str)
    

In [ ]:
#get an overview of the data we just pulled and drop unnecessary ones
for df in df_dict:
    print(df)
    print(df_dict[df].shape)
    display(df_dict[df].head())

In [ ]:
def check_null_cols(df_dict):
    #TODO: write a docstring
    
    for df in df_dict:
        print(df)
        display(pd.DataFrame((np.sum(df_dict[df].isnull())/df_dict[df].shape[0])))

check_null_cols(df_dict)

In [ ]:
# drop tags from course-meta... too many nulls
# TODO: also drop entries from course and course meta with catch < max(catch)
# for the others just drop rows missing value

# to drop:
#  - all earlier entries of a course - we only want the most recent one
#      solution from : https://stackoverflow.com/a/51820845
#  - columns we won't need thereafter
#  - potentially remaining duplicates
course_meta_df = df_dict['course-meta'].\
                    sort_values(by = ['id', 'catch']).\
                    drop_duplicates(subset=['id'], keep = 'last').\
                    drop(columns = ['tag', 'catch']).\
                    dropna(subset = ['firstClear'], axis = 0).\
                    drop_duplicates()

course_df = df_dict['courses'].\
                drop(columns = ['thumbnail', 'image', 'title']).\
                dropna(subset = ['maker'], axis = 0).\
                drop_duplicates()

course_complete_df = course_meta_df.merge(course_df, on = 'id')

players_df = df_dict['players'].drop(columns = ['image', 'name']).\
                rename(columns= {'id' : 'player_id'}).\
                drop_duplicates()



In [ ]:
# # group_test =\
# course_meta_df.loc[
#     course_meta_df.catch == course_meta_df.catch.max()].index

In [ ]:
# df_dict['course-meta'].sort_values(['id', 'catch'])\
#         #.drop_duplicates(subset = ['id'], keep = 'last')

In [ ]:
#get player flags
course_complete_df = course_complete_df.merge(players_df,
                                             how = 'left',
                                             left_on = 'maker',
                                             right_on = 'player_id')

print(course_complete_df.shape)
print(course_complete_df.columns)

In [ ]:
course_complete_df.head()

In [ ]:
country_counts = course_complete_df.flag.value_counts()
difficulty_counts = course_complete_df.difficulty.value_counts() 

In [ ]:
(country_counts/course_complete_df.shape[0]).sort_values().plot(kind = 'bar')
plt.title("Share of Levels Created by Country");

In [ ]:
(difficulty_counts/course_complete_df.shape[0]).sort_values().plot(kind = 'bar')
plt.title("Share of Levels Created by Difficulty");

In [ ]:
print(course_complete_df.creation.max(), course_complete_df.creation.min())

In [ ]:
course_complete_df.flag.unique()

In [ ]:
players_df.flag.unique()

In [ ]:
# extract number of courses created and add as extra column
course_complete_df['courses_by_maker'] = course_complete_df.groupby('maker').transform('count').id

# operationalize age of course relative to earliest date in df in days
course_complete_df['creation'] = pd.to_datetime(course_complete_df.creation)
course_complete_df['normalized_age'] = (course_complete_df.creation - course_complete_df.creation.min()).dt.days

In [ ]:
course_clean_df = course_complete_df.drop(columns = 
                ['firstClear', 'id', 'maker', 'player_id', 'clears', 'creation'])

# also drop clears... the info is not needed as we have 
# clears = attempts*(clearRate/100)

# we're dropping maker here although it might be useful...
# use somewhere else in analysis
# we should also extract creation in a meaningful way.
course_clean_df.head()

In [ ]:
# split columns into num_ and cat_

cat_cols = ['difficulty', 'gameStyle', 'flag']
num_cols = [col for col in course_clean_df.columns if col not in cat_cols]
# gives the right result and doesnt require us to split the df

In [ ]:
#taken from lectures
def get_dummy_df(df, cat_cols, dummy_na=True):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    
    for column in cat_cols:
        
        try:
            df = pd.concat([df.drop(column, axis = 1), pd.get_dummies(df[column], dummy_na=dummy_na, prefix = column, 
                                                                     prefix_sep = "_", drop_first = True)], axis = 1)
        except:
            continue
            
    return df
            

    

In [ ]:
course_final_df = get_dummy_df(course_clean_df, cat_cols, False) # already checked, there are no NaNs left
course_final_df.head()

In [ ]:
# prepare for linreg

y_stars = course_final_df.stars
y_tweets = course_final_df.tweets

X_stars = course_final_df.drop(columns = ['stars'])
X_tweets = course_final_df.drop(columns = ['tweets'])

X_stars_train, X_stars_test, y_stars_train, y_stars_test = train_test_split(X_stars, y_stars, 
                                                                            test_size = .33, random_state = 23)

X_tweets_train, X_tweets_test, y_tweets_train, y_tweets_test = train_test_split(X_tweets, y_tweets, 
                                                                            test_size = .33, random_state = 23)

lm_stars = LinearRegression().fit(X_stars_train,y_stars_train)
lm_tweets = LinearRegression().fit(X_tweets_train,y_tweets_train)

y_stars_train_preds = lm_stars.predict(X_stars_train)
y_stars_test_preds = lm_stars.predict(X_stars_test)

y_tweets_train_preds = lm_tweets.predict(X_tweets_train)
y_tweets_test_preds = lm_tweets.predict(X_tweets_test)

train_score_stars = r2_score(y_stars_train, y_stars_train_preds)
test_score_stars = r2_score(y_stars_test, y_stars_test_preds)

train_score_tweets = r2_score(y_tweets_train, y_tweets_train_preds)
test_score_tweets = r2_score(y_tweets_test, y_tweets_test_preds)

In [ ]:
out = \
'R2 Training (Stars): \t' + str(train_score_stars) + '\n' +\
'R2 Testing (Stars): \t' + str(test_score_stars) + '\n' +\
'R2 Training (Tweets): \t' + str(train_score_tweets) + '\n' +\
'R2 Testing (Tweets): \t' + str(test_score_tweets) + '\n'

In [ ]:
print(out)

In [ ]:
print(lm_stars.coef_)
print(lm_tweets.coef_)

In [ ]:
def coef_weights(model, coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = model.coef_
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df


In [ ]:
#Use the function
coef_stars_df = coef_weights(lm_stars, lm_stars.coef_, X_stars_train)

#A quick look at the top results
coef_stars_df.head(20)

In [ ]:
coef_tweets_df = coef_weights(lm_tweets, lm_tweets.coef_, X_tweets_train)

#A quick look at the top results
coef_tweets_df.head(20)

In [ ]:
course_clean_df.gameStyle.unique()